# __Search image by image__

## Preface

- Tutorial Difficulty : ★☆☆☆☆
- 7 min read
- Languages : [SQL](https://en.wikipedia.org/wiki/SQL) (100%)
- File location : tutorial_en/thanosql_search/search_image_by_image.ipynb
- References : [MNIST DataSet](http://yann.lecun.com/exdb/mnist/), [A Simple Framework for Contrastive Learning of Visual Representations](https://arxiv.org/abs/2002.05709)

## Introduction to the tutorial

<div class="admonition note">
    <h4 class="admonition-title">Understanding image digitization techniques</h4>
    <p>Images are high-dimensional data (height x width x channel [RGB] x color intensity), which means nothing if the information for each pixel is randomly generated. In other words, each pixel can only be recognized as an image if it has a specific pattern associated with the surrounding pixel. This means that an image can be represented on a lower-dimensional characteristic vector than it really is. Recently, studies using artificial intelligence to numericalize and express each image in a low-dimensional space according to the similarity of the meaning of each image have been conducted in various ways, and these have been used in various names such as image digitization, vectorization, and embedding.</p>
</div>

There are several ways to define the similarity of an image. It could be that the colors are similar, the objects in the image are similar, or they have the same meaning, such as handwriting. Although it is difficult to give an exact definition of a similar image, artificial intelligence learns and quantifies the general characteristics of an image.

ThanoSQL uses [Self-Supervised Learning Model](https://en.wikipedia.org/wiki/Self-supervised_learning) to input images and retrieve similar images from DB . When users upload their images to ThanoSQL's DB, they learn by themselves by placing similar images close and other images far away through an artificial intelligence algorithm. You can learn a general representation of an image from a dataset with no correct answer, fine-tune it to an image with a small amount of target values, and use it for classification or regression tasks.

In addition, ThanoSQL uses artificial intelligence algorithms to quantify datasets. The digitized data is stored in the DB column and used to search for similar images by calculating the similarity(distance) between images.

__The following is an example of the ThanoSQL similar image search algorithm.__

- When a user's favorite image is input, it searches for works of art with a similar feeling among the works of art stored in the DB and recommends them to the user.
- Find similar images in an album of thousands of photos.
- You can create your own search engine or artificial intelligence model by storing the numerical value of the image in ThanoSQL's DB, and using the ThanoSQL Auto-ML regression/classification prediction model.
 
<div class="admonition note">
    <h4 class="admonition-title">In this tutorial</h4>
    <p>👉 This tutorial will use <mark style="background-color:#FFD79C">MNIST handwriting dataset</mark>. Each image is a fixed size(28x28 = 784 pixels) with a value between 0 and 1 and provides a number from 0 to 9 written by many people with the correct answer. It consists of 1,000 learning datasets and 200 testing datasets.</p>
</div>
    
Try creating a model that uses ThanoSQL to input handwriting data and retrieves images similar to the input image in the DB.

[![IMAGE](https://docs.thanosql.ai/img/thanosql_search/search_image_by_image/simclr_img7.png "MNIST data") ](https://docs.thanosql.ai/img/thanosql_search/search_image_by_image/simclr_img7.png)

## __0. Prepare Dataset__

To use the query syntax of ThanoSQL, you must create an API token and run the query below, as mentioned in the [ThanoSQL Workspace](https://docs.thanosql.ai/en/getting_started/how_to_use_ThanoSQL/#5-thanosql-workspace).

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<Issued_API_TOKEN>

### __Prepare Dataset__

In [ ]:
%%thanosql
GET THANOSQL DATASET mnist_data
OPTIONS (overwrite=True)

<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" Use the query syntax to save the desired dataset to the workspace. </li>
        <li>"<strong>OPTIONS</strong>"Specifies the option to use for <strong>GET THANOSQL DATASET</strong> via query syntax.
        <ul>
            <li>"overwrite" : Set whether to overwrite if a dataset with the same name exists. If True, the old dataset is replaced with the new dataset (True|False, DEFAULT : False) </li>
        </ul>
        </li>
    </ul>
</div>

In [ ]:
%%thanosql
COPY mnist_train 
OPTIONS (overwrite=True)
FROM "thanosql-dataset/mnist_data/mnist_train.csv"

In [ ]:
%%thanosql
COPY mnist_test 
OPTIONS (overwrite=True)
FROM "thanosql-dataset/mnist_data/mnist_test.csv"

<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>Use the "<strong>COPY</strong>" query syntax to specify the dataset name to store in the DB. </li>
        <li>"<strong>OPTIONS</strong>" Specifies the options to use for <strong>COPY</strong> query syntax.
        <ul>
            <li>"overwrite" : Set whether or not a dataset with the same name can be overwritten if it exists on the DB. If True, the existing dataset is changed to the new dataset (True|False, DEFAULT: False) </li>
        </ul>
        </li>
    </ul>
</div>

## __1. Check Dataset__

To create a handwriting classification model, use the <mark style="background-color:#FFEC92">mnist_train</mark> table stored in ThanoSQL [DB](https://en.wikipedia.org/wiki/Database). The <mark style="background-color:#FFEC92">mnist_train</mark> table contains the path, file name, and label information that contains the <mark style="background-color:#FFD79C">MNIST</mark> image files. Run the query statement below and check the contents of the table.

In [ ]:
%%thanosql
SELECT * 
FROM mnist_train 
LIMIT 5

<div class="admonition note">
    <h4 class="admonition-title">Understanding the Data Table</h4>
    <p>The <mark style="background-color:#FFEC92">mnist_train</mark> table contains the following information: The "6782.jpg" image file is a handwritten image with the number 5.</p>
    <ul>
        <li><mark style="background-color:#D7D0FF">image_path</mark>: image path</li>
        <li><mark style="background-color:#D7D0FF">filename</mark>: file name</li>
        <li><mark style="background-color:#D7D0FF">label</mark>: image label</li>
    </ul>
</div>

## __2. Creating an Image Numerical Model__

Create an image quantification model using the <mark style="background-color:#FFEC92">mnist_train</mark> table from the previous step. Execute the query syntax below to create a model named <mark style="background-color:#E9D7FD">my_image_search_model</mark>.  
(Estimated time required for query execution: 1 min)

In [ ]:
%%thanosql
BUILD MODEL my_image_search_model
USING SimCLR
OPTIONS (
    image_col="image_path",
    max_epochs=1,
    overwrite=True
    )
AS 
SELECT * 
FROM mnist_train

<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>Create and train a model called <mark style="background-color:#E9D7FD">mnist_model</mark> using the "<strong>BUILD MODEL</strong>" query syntax.</li>
        <li>The "<strong>USING</strong>" query syntax specifies that the <mark style="background-color:#E9D7FD">SimCLR</mark> model should be used as the base model.</li>
        <li>"<strong>OPTIONS</strong>" Specify the options to use to create the model through the query syntax.
        <ul>
            <li>"image_col" : Column containing the path of the image in the data table (Default: "<mark style="background-color:#D7D0FF">image_path</mark>")</li>
            <li>"max_epochs" : Number of datasets learned to generate image quantization models</li>
            <li>"overwrite" : Set whether or not a model with the same name exists. If true, the existing model is changed to a new model (True|False, DEFAULT: False) </li>
        </ul>
        </li>
    </ul>
</div>

Run the following "__CONVERT USING__" query syntax to digitize the images `mnist_test`. The quantized results are stored in a column named <mark style="background-color:#D7D0FF">my_image_search_model_simclr</mark> in the table `mnist_test`.

In [ ]:
%%thanosql
CONVERT USING my_image_search_model
OPTIONS (
    table_name= "mnist_test",
    image_col="image_path"
    )
AS 
SELECT * 
FROM mnist_test

<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>The "<strong>CONVERT USING</strong>" query syntax uses <code>my_image_search_model</code> as an algorithm for image quantification.   </li>
        <li>The "<strong>OPTIONS</strong>" query syntax defines the variables required for image quantification.
        <ul>
            <li>"table_name" : Defines the table name to be stored in the ThanoSQL DB. </li>
            <li>"image_col" : Column containing the path of the image in the data table (default: "image_path")</li>
        </ul>
        </li>
    </ul>
</div>

## __3. Search for similar images using image quantization models__

This step uses the <mark style="background-color:#E9D7FD">my_image_search_model</mark> image quantization model and the test table to search for images similar to the "923.jpg" image file (handwriting 8).

<a href="https://docs.thanosql.ai/img/thanosql_search/search_image_by_image/simclr_img8.png">
    <img alt="IMAGE" src="https://docs.thanosql.ai/img/thanosql_search/search_image_by_image/simclr_img8.png" style="width:100px">
</a>

<p style="text-align:center">923.jpg Image File </p>

In [ ]:
%%thanosql
SEARCH IMAGE images='thanosql-dataset/mnist_data/test/923.jpg' 
USING my_image_search_model 
AS
SELECT * 
FROM mnist_test

<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>SEARCH IMAGE [images|audio|videos]</strong>" query syntax defines the image|audio|video file you want to search for.  <br></li>
        <li>"<strong>USING</strong>" defines the model used for image quantification.<br></li>
        <li>The "<strong>AS</strong>" query syntax defines the embedding table to use for searches. Use the <code>mnist_embds</code> table </li>
    </ul>
</div>

Run the following query to output the "__SEARCH__" result using the "__PRINT__" query syntax in ThanoSQL to output the top four most similar. We've only done a little bit of learning, but you can see that it's outputting an image similar to 8.

In [ ]:
%%thanosql
PRINT IMAGE 
AS (
    SELECT image_path, my_image_search_model_simclr_similarity1 
    FROM (
        SEARCH IMAGE images='thanosql-dataset/mnist_data/test/923.jpg' 
        USING my_image_search_model 
        AS 
        SELECT * 
        FROM mnist_test
        )
    ORDER BY my_image_search_model_simclr_similarity1 DESC 
    LIMIT 4
    )

<div class="admonition danger">
    <h4 class="admonition-title">Note</h4>
    <p>The basic learning options of the image similarity search algorithm are learned to recognize the image as the same regardless of the image's left-right inversion, color change, etc. This is because a dog's picture should be recognized as a dog even if it is flipped or changed in color. If color changes are important, such as clothing images, or if vertical and horizontal twists are important, such as numbers, the options should be changed when learning. This tutorial shows the characteristics of these image similarity searches.</p>
</div>

## __4. In Conclusion__

In this tutorial, we used the `MNIST` handwriting dataset to perform image quantification and similar image search based on quantification results. In this tutorial, we explained the operation rather than the accuracy of image similarity. Image quantification models can improve accuracy by adding precise tuning and small amounts of correct answers to each image dataset during learning. You can create your own image quantization model to add search capabilities to various types of unstructured datasets and deploy your own model using Auto-ML techniques.
<br>
The next step is to explore the various "__OPTIONS__" query syntax and learning methods of image quantification models. If you want to learn more about how to build your own accurate image conversion model, go ahead with the following tutorials.

- [How to Upload to ThanoSQL DB](https://docs.thanosql.ai/en/how-to_guides/ThanoSQL_connecting/data_upload/)
- [Creating an Intermediate Similar Image Search Model]

<div class="admonition tip">
    <h4 class="admonition-title">Inquiries about deploying a model for your own service</h4>
    <p>If you have any difficulties in creating your own model using ThanoSQL or applying it to the service, please feel free to contact us below😊</p>
    <p>For inquiries about building a similar image search model: contact@smartmind.team</p>
</div>